In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data.core import *
from local.data.external import *
from local.notebook.showdoc import show_doc

In [ ]:
#default_exp text.core

# Text core

> Basic function to preprocess text before assembling it in a `DataBunch`.

In [ ]:
#export 
from multiprocessing import Process, Queue
import spacy,html
from spacy.symbols import ORTH

## Preprocessing rules

The following are rules applied to texts before or after it's tokenized.

In [ ]:
#export
#special tokens
UNK, PAD, BOS, EOS, FLD, TK_REP, TK_WREP, TK_UP, TK_MAJ = "xxunk xxpad xxbos xxeos xxfld xxrep xxwrep xxup xxmaj".split()

In [ ]:
#export
_re_spec = re.compile(r'([/#\\])')

def spec_add_spaces(t):
    "Add spaces around / and #"
    return _re_spec.sub(r' \1 ', t)

In [ ]:
test_eq(spec_add_spaces('#fastai'), ' # fastai')
test_eq(spec_add_spaces('/fastai'), ' / fastai')
test_eq(spec_add_spaces('\\fastai'), ' \\ fastai')

In [ ]:
#export
_re_space = re.compile(' {2,}')

def rm_useless_spaces(t):
    "Remove multiple spaces"
    return _re_space.sub(' ', t)

In [ ]:
test_eq(rm_useless_spaces('a  b   c'), 'a b c')

In [ ]:
#export
_re_rep = re.compile(r'(\S)(\1{3,})')

def replace_rep(t):
    "Replace repetitions at the character level: cccc -> TK_REP 4 c"
    def _replace_rep(m):
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    return _re_rep.sub(_replace_rep, t)

It starts replacing at 4 repetitions of the same character or more.

In [ ]:
test_eq(replace_rep('aaa'), 'aaa')
test_eq(replace_rep('aaaa'), f' {TK_REP} 4 a ')

In [ ]:
#export
_re_wrep = re.compile(r'(?:\s|^)(\w+)\s+((?:\1\s+){2,})\1(\s|\W|$)')

#hide

Matches any word repeated at least four times with spaces between them
```
(?:\s|^)          Non-catching group with either a whitespace character or the beginning of text
(\w+)             Catching group of any alphanumeric character
\s+               One or more whitespace
((?:\1\s+){2,})   Catching group of a repetition of two or more times \1 followed by one or more whitespace
\1                Occurence of \1
(\s|\W|$)         Catching group of last whitespace, non alphanumeric character or end of text
```

In [ ]:
#export
def replace_wrep(t):
    "Replace word repetitions: word word word word -> TK_WREP 4 word"
    def _replace_wrep(m):
        c,cc,e = m.groups()
        return f' {TK_WREP} {len(cc.split())+2} {c} {e}'
    return _re_wrep.sub(_replace_wrep, t)

It starts replacing at 4 repetitions of the same word or more.

In [ ]:
test_eq(replace_wrep('ah ah'), 'ah ah')
test_eq(replace_wrep('ah ah ah ah'), f' {TK_WREP} 4 ah ')
test_eq(replace_wrep('ah ah   ah  ah'), f' {TK_WREP} 4 ah ')
test_eq(replace_wrep('ah ah ah ah '), f' {TK_WREP} 4 ah  ')
test_eq(replace_wrep('ah ah ah ah.'), f' {TK_WREP} 4 ah .')
test_eq(replace_wrep('ah ah ah ahi'), f'ah ah ah ahi')

In [ ]:
#export
def fix_html(x):
    "Various messy things we've seen in documents"
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace('nbsp;', ' ').replace(
        '#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace('<br />', "\n").replace(
        '\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(' @-@ ','-')
    return html.unescape(x)

In [ ]:
test_eq(fix_html('#39;bli#146;'), "'bli'")
test_eq(fix_html('Sarah amp; Duck'), 'Sarah & Duck')
test_eq(fix_html('a nbsp; #36;'), 'a   $')
test_eq(fix_html('\\" <unk>'), f'" {UNK}')
test_eq(fix_html('quot;  @.@  @-@ '), "' .-")
test_eq(fix_html('<br />text\\n'), '\ntext\n')

In [ ]:
#export
_re_all_caps = re.compile(r'(\s|^)([A-Z]+[^a-z\s]*)(?=(\s|$))')

#hide

Catches any word in all caps, even with ' or - inside
```
(\s|^)        Catching group with either a whitespace or the beginning of text
([A-Z]+       Catching group with one capitalized letter or more...
[^a-z\s]*)    ...followed by anything that's non lowercase or whitespace
(?=(\s|$))    Look ahead for a space of end of text
```
The look ahead is there to not move the pointer ahead of the next space in case we have consecutive words in all caps.

In [ ]:
#export
def replace_all_caps(t):
    "Replace tokens in ALL CAPS by their lower version and add `TK_UP` before."
    def _replace_all_caps(m):
        tok = f'{TK_UP} ' if len(m.groups()[1]) > 1 else ''
        return f"{m.groups()[0]}{tok}{m.groups()[1].lower()}"
    return _re_all_caps.sub(_replace_all_caps, t)

In [ ]:
test_eq(replace_all_caps("I'M SHOUTING"), f"{TK_UP} i'm {TK_UP} shouting")
test_eq(replace_all_caps("I'm speaking normally"), "I'm speaking normally")
test_eq(replace_all_caps("I am speaking normally"), "i am speaking normally")

In [ ]:
#export
_re_maj = re.compile(r'(\s|^)([A-Z][^A-Z\s]*)(?=(\s|$))')

#hide

Catches any capitalized word
```
(\s|^)       Catching group with either a whitespace or the beginning of text
([A-Z]       Catching group with exactly one capitalized letter...
[^A-Z\s]*)   ...followed by anything that's not uppercase or whitespace
(?=(\s|$))   Look ahead for a space of end of text
```
The look ahead is there to not move the pointer ahead of the next space in case we have consecutive words in all caps.

In [ ]:
#export
def replace_maj(t):
    "Replace tokens in ALL CAPS by their lower version and add `TK_UP` before."
    def _replace_maj(m):
        tok = f'{TK_MAJ} ' if len(m.groups()[1]) > 1 else ''
        return f"{m.groups()[0]}{tok}{m.groups()[1].lower()}"
    return _re_maj.sub(_replace_maj, t)

In [ ]:
test_eq(replace_maj("Jeremy Howard"), f'{TK_MAJ} jeremy {TK_MAJ} howard')
test_eq(replace_maj("I don't think there is any maj here"), ("i don't think there is any maj here"),)

In [ ]:
#export
def lowercase(t):
    "Converts `t` to lowercase"
    return t.lower()

In [ ]:
defaults.text_proc_rules = [fix_html, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces,
                            replace_all_caps, replace_maj, lowercase]

## Tokenizing

A tokenizer is a class that must implement a `pipe` method. This `pipe` method receives a generator of texts and must return a generator with their tokenized versions. Here is the most basic example:

In [ ]:
class BaseTokenizer():
    "Basic tokenizer that just splits on spaces"
    def __init__(self, **kwargs): pass
    def pipe(self, items): 
        for t in items: yield t.split()

In [ ]:
tok = BaseTokenizer()
for t in tok.pipe(["This is a text"]): test_eq(t, ["This", "is", "a", "text"])

In [ ]:
def apply_rules(items, rules):
    "Returns a generator that apply `rules`  to `items`"
    for o in items: yield compose(*rules)(o)

In [ ]:
for t in apply_rules(["This is a text"], [replace_maj]): test_eq(t, f"{TK_MAJ} this is a text")

In [ ]:
def tokenize1(text, tok_func=SpacyTokenizer, rules=None, **tok_kwargs):
    "Tokenize one `text` with an instance of `tok_func` and some `rules`"
    rules = L(ifnone(rules, defaults.text_proc_rules))
    tokenizer = tok_func(**tok_kwargs)
    for tok in tokenizer.pipe(apply_rules([text], rules)): return tok

In [ ]:
test_eq(tokenize1("This is a text"), [TK_MAJ, 'this', 'is', 'a', 'text'])

In [ ]:
def tok_items(items, tok_func, rules, output_func, output_queue, data_queue=None, **tok_kwargs):
    'Tokenize `items` with an instance of `tok_func` and some `rules`'
    tokenizer = tok_func(**tok_kwargs)
    if data_queue: counts = Counter()
    for o,tok in zip(items, tokenizer.pipe(apply_rules(items, rules))):
        output_queue.put(output_func(o, tok))
        if data_queue: counts.update(Counter(tok))
    if data_queue: data_queue.put(counts)

The main function that will be called during one of the processes handling tokenization. It will create an instance of a tokenizer with `tok_func` and `tok_kwargs`, then iterate through the `items`, apply them `rules`, tokenize them, then apply `output_func` to the original item and the tokens and put the result in `output_queue`.

If a `data_queue` is passed, we count the different tokens and return the Counter in it at the end, to save the vocab.

In [ ]:
def launch_process(items, tok_func, rules, output_func):#Just for testing
    output_queue = Queue(1)
    process = Process(target=tok_items,
                      args=(items, tok_func, rules, output_func, output_queue))
    process.start()
    for _ in range(2): print(output_queue.get())
    process.join()

texts = ["this is a text", "this is another text"]
rules = [lambda i: texts[i]]
output_func = lambda i,t: t
test_stdout(lambda: launch_process([0,1], BaseTokenizer, rules, output_func),
           """['this', 'is', 'a', 'text']
['this', 'is', 'another', 'text']""")

### Tokenize texts in files

Helper function to create the same directory structure as in a given folder.

In [ ]:
def create_folders(path, output_dir, include=None):
    output_dir = Path(output_dir)
    os.makedirs(output_dir, exist_ok=True)
    for i,(p,d,f) in enumerate(os.walk(path)): # returns (dirpath, dirnames, filenames)
        if include is not None and i==0: d[:] = [o for o in d if o in include]
        else:                            d[:] = [o for o in d if not o.startswith('.')]
        for x in d: os.makedirs(output_dir/(Path(p)/Path(x)).relative_to(path), exist_ok=True)

Preprocessing function for texts in filenames. Tokenized texts will be saved in a similar fashion in a directory suffixed with `_tok` in the parent folder of `path` (override with `output_dir`).

In [ ]:
def read_text(fname):
    with open(fname, 'r') as f: return f.read()

In [ ]:
SEP = '▁'

In [ ]:
fname = path/'labels.csv'

In [ ]:
fname.suffix

In [ ]:
def tok_folder(path, extensions=['.txt'], include=None, output_dir=None, n_workers=4,
               pre_rules=None, post_rules=None, tok_func=SpacyTokenizer, **tok_kwargs):
    path = Path(path)
    fnames = get_files(path, extensions=extensions, recurse=True, include=include)
    output_dir = Path(ifnone(output_dir, path.parent/f'{path.name}_tok'))
    create_folders(path, output_dir, include=include)
    pre_rules = [read_text] + listify(ifnone(pre_rules, default_pre_rules.copy()))
    post_rules = listify(ifnone(post_rules, default_post_rules.copy()))
    
    output_queue,data_queue = Queue(maxsize=n_workers),Queue(maxsize=n_workers)
    def _output(o, tok):
        out = output_dir/o.relative_to(path)
        with open(out, 'w') as f: f.write(SEP.join(tok))
        with open(out.parent/f'{out.stem}.len', 'w') as f: f.write(str(len(tok)))
        return 1
            
    processes = [Process(target=tok_items,
                         args=(batch, tok_func, pre_rules, post_rules, _output, output_queue),
                         kwargs={'data_queue': data_queue, **tok_kwargs})
                 for i,batch in enumerate(np.array_split(fnames, n_workers))]
    
    for p in processes: p.start()
    counter = Counter()
    for _ in progress_bar(fnames, leave=False): _ = output_queue.get()
    for _ in processes: counter.update(data_queue.get())
    for p in processes: p.join()
    pickle.dump(counter, open(output_dir/'counter.pkl','wb'))

In [ ]:
path = untar_data(URLs.IMDB)

In [ ]:
# test
fnames = get_files(path, extensions=['.txt'], recurse=True, include=['train', 'test', 'unsup'])
tok_path = path.parent/'imdb_tok'
assert tok_path.exists()
#Take one file randomly
idx = random.randint(0, len(fnames)-1)
#Check we have the corresponding tokenized version...
tok_fname = tok_path/(fnames[idx].relative_to(path))
assert tok_fname.exists()
text = read_text(fnames[idx])
tok = tokenize1(text)
assert SEP.join(tok) == read_text(tok_fname)
len_fname = tok_fname.parent/f'{tok_fname.stem}.len'
assert len(tok) == int(read_text(len_fname))

When text is in a dataframe, we need to merge the text columns, and maybe mark_fields.

In [ ]:
def join_texts(idx, df, mark_fields=False):
    return ' '.join([(f'{FLD} {i} ' if mark_fields else '') + t for i,t in enumerate(df.iloc[int(idx)].values)])

Preprocessing function for texts in a dataframe. Tokenized texts will be put in a similar dataframe with just one column of texts and the other columns the same.

In [ ]:
def tok_df(df, text_cols, n_workers=4, pre_rules=None, post_rules=None, mark_fields=None, 
           tok_func=SpacyTokenizer, **tok_kwargs):
    text_cols = listify(text_cols)
    mark_fields = ifnone(mark_fields, len(listify(text_cols)) > 1)
    pre_rules = listify(ifnone(pre_rules, default_pre_rules.copy()))
    pre_rules = [partial(join_texts, df=df[text_cols], mark_fields=mark_fields)] + pre_rules
    post_rules = listify(ifnone(post_rules, default_post_rules.copy()))
    
    output_queue,data_queue = Queue(maxsize=n_workers),Queue(maxsize=n_workers)
    def _output(o, tok): return (o,tok)
            
    processes = [Process(target=tok_items,
                         args=(batch, tok_func, pre_rules, post_rules, _output, output_queue),
                         kwargs={'data_queue': data_queue, **tok_kwargs})
                 for i,batch in enumerate(np.array_split(range(len(df)), n_workers))]
    
    for p in processes: p.start()
    lengths,outputs,counter = np.zeros(len(df)),np.zeros(len(df), dtype=np.object),Counter()
    for _ in progress_bar(range(len(df)), leave=False): 
        i,tok = output_queue.get()
        lengths[i],outputs[i] = len(tok),SEP.join(tok)
    for _ in processes: counter.update(data_queue.get())
    for p in processes: p.join()
    
    other_cols = [c for c in df.columns if c not in text_cols]
    res = df[other_cols].copy()
    res['text'],res['text_lengths'] = outputs,lengths
    return res, counter

In [ ]:
# test
path = untar_data(URLs.IMDB_SAMPLE)
df = pd.read_csv(path/'texts.csv')
out,cnt = tok_df(df, text_cols='text')
test_eq(set(out.columns),set(list(df.columns)+['text_lengths']))
idx = random.randint(0, len(df)-1)
text = df['text'][idx]
tok = tokenize1(text)
test_eq(SEP.join(tok), out['text'][idx])
test_eq(len(tok), out['text_lengths'][idx])
#With two fields, mark fields become true by default
df['text1'] = df['text']
out,cnt = tok_df(df, text_cols=['text', 'text1'])
idx = random.randint(0, len(df)-1)
text = f"{FLD} 0 {df['text'][idx]} {FLD} 1 {df['text1'][idx]}"
tok = tokenize1(text)
test_eq(SEP.join(tok), out['text'][idx])
test_eq(len(tok), out['text_lengths'][idx])

In [ ]:
def tok_csv(fname, text_cols, outname=None, n_workers=4, pre_rules=None, post_rules=None, 
            mark_fields=None, tok_func=SpacyTokenizer, header='infer', chunksize=None, **tok_kwargs):
    df = pd.read_csv(fname, header=header, chunksize=chunksize)
    outname = Path(ifnone(outname, fname.parent/f'{fname.stem}_tok.csv'))
    kwargs = dict(n_workers=n_workers, pre_rules=pre_rules, post_rules=post_rules, 
                  mark_fields=mark_fields, tok_func=tok_func, **tok_kwargs)
    if chunksize is None:
        out,cnt = tok_df(df, text_cols, **kwargs)
        out.to_csv(outname, header=header, index=False)
    else:
        cnt = Counter()
        for i,dfp in enumerate(df):
            out,c = tok_df(dfp, text_cols, **kwargs)
            out.to_csv(outname, header=header if i==0 else None, index=False, mode='w' if i==0 else 'a')
            cnt.update(c)
    pickle.dump(cnt, open(outname.parent/'counter.pkl', 'wb'))

In [ ]:
#test
path = untar_data(URLs.IMDB_SAMPLE)
tok_csv(path/'texts.csv', 'text')
assert (path/'texts_tok.csv').exists()
df = pd.read_csv(path/'texts.csv')
df_tok = pd.read_csv(path/'texts_tok.csv')
idx = random.randint(0, len(df)-1)
text = df['text'][idx]
tok = tokenize1(text)
test_eq(SEP.join(tok), df_tok['text'][idx])
test_eq(len(tok), df_tok['text_lengths'][idx])

In [ ]:
#test
path = untar_data(URLs.IMDB_SAMPLE)
tok_csv(path/'texts.csv', 'text', chunksize=500)
assert (path/'texts_tok.csv').exists()
df = pd.read_csv(path/'texts.csv')
df_tok = pd.read_csv(path/'texts_tok.csv')
test_eq(len(df_tok), len(df))
idx = random.randint(0, len(df)-1)
text = df['text'][idx]
tok = tokenize1(text)
test_eq(SEP.join(tok), df_tok['text'][idx])
test_eq(len(tok), df_tok['text_lengths'][idx])

In [ ]:
class SpacyTokenizer():
    "Spacy tokenizer for `lang`"
    def __init__(self, lang='en', special_toks=None, batch_size=5000):
        special_toks = ifnone(special_toks, default_spec_tok)
        self.nlp = spacy.blank(lang, disable=["parser", "tagger", "ner"])
        for w in default_spec_tok: self.nlp.tokenizer.add_special_case(w, [{ORTH: w}])
        self.batch_size=batch_size
    
    def pipe(self, items):
        for doc in self.nlp.pipe(items, batch_size=self.batch_size):
            yield [d.text for d in doc]